In [1]:
from google.colab import drive
# drive.mount('/content/drive/MyDrive/IITP/sohyun/creditcard_prediction/data')
drive.mount('/content/drive')

%cd drive/MyDrive/IITP/sohyun/TimeSeriesAnomaly/data/modify

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1j1N0u5t0l99N_wfSd5UZvnhugzn5g_NC/TimeSeriesAnomaly/data/modify


In [ ]:
# !pip install wandb -qqq
# import wandb
# wandb.login()

In [2]:
import matplotlib.pyplot as plt
import easydict
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import random
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
import time
import math

import seaborn as sns
from pylab import rcParams
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn import metrics
import copy

## Data 

In [3]:
#---# LOAD npy file #---#
Fu_20_normal = np.load('Fu_20_normal.npy')
Fu_21_normal = np.load('Fu_21_normal.npy')
Fu_21_abnormal = np.load('Fu_21_abnormal.npy')
Fu_22_normal = np.load('Fu_22_normal.npy')
Fu_22_abnormal = np.load('Fu_22_abnormal.npy')

Fu_20_normal_10 = np.load('Fu_20_normal_10.npy')
Fu_21_normal_10 = np.load('Fu_21_normal_10.npy')
Fu_21_abnormal_10 = np.load('Fu_21_abnormal_10.npy')
Fu_22_normal_10 = np.load('Fu_22_normal_10.npy')
Fu_22_abnormal_10 = np.load('Fu_22_abnormal_10.npy')

# import sys
# np.set_printoptions(threshold=sys.maxsize) # print all

#---# 확인용 #---#
# plt.figure(figsize=(30,5))
# plt.plot(Fu_22_abnormal_10)

In [5]:
class MyDataset(Dataset):
  def __init__(self, config, mode="train"):
    # 최종 목표 : path
    # self.data_path_list = data_path_list
    # self.data_list = [_load_data() for data_path in self.data_path_list] 
    self.mode = mode
    self.config = config
    self.device = self.config.device
    self.train_list = [Fu_22_normal_10, Fu_21_normal_10, Fu_20_normal_10]
    self.test_list = [Fu_22_abnormal_10]
    
    # scaling
    self.scaled_train_list, self.scaled_test = self._scale()
    
    # sliding window & concat
    self.slided_train_list = [self._sliding_window(data) for data in self.scaled_train_list]
    self.train = np.concatenate(self.slided_train_list, axis=0)
    
    # split
    y = [0 for _ in range(len(self.train))]
    self.x_train, self.x_val, _, _ = train_test_split(self.train, y, test_size=0.2, random_state=self.config.seed)
    self.x_test = self._sliding_window(self.scaled_test)
    
  def __getitem__(self, index):
    if self.mode == "train":
      self.x = self.x_train[index]
    elif self.mode == "test":
      self.x = self.x_test[index]
    elif self.mode == "val":
      self.x = self.x_val[index]
    return torch.tensor(self.x, dtype=torch.float32)
      
  def __len__(self):
    if self.mode == "train":
      self.x = self.x_train
    elif self.mode == "test":
      self.x = self.x_test
    elif self.mode == "val":
      self.x = self.x_val
    return len(self.x)
      
  def _scale(self):
    '''
    input
    output
    '''
    train = np.concatenate(self.train_list, axis=0)
    test = self.test_list[0]
    total = np.concatenate([train, test])

    scaler = StandardScaler()
    total_scaled = scaler.fit_transform(total.reshape(-1, 1)).squeeze()
    train_scaled = scaler.transform(train.reshape(-1,1)).squeeze()
    test_scaled = scaler.transform(test.reshape(-1,1)).squeeze()

    train_list = []
    split_idx1, split_idx2 = len(self.train_list[0]), len(self.train_list[1]) 

    train_list.append(train_scaled[:split_idx1])
    train_list.append(train_scaled[split_idx1:split_idx1+split_idx2])
    train_list.append(train_scaled[split_idx1+split_idx2:])

    return train_list, test_scaled

  def _sliding_window(self, arr):
    start_pt = 0
    total_data = []
    while(True):
      if len(arr) < (start_pt + config.window_size) : break
      data = arr[start_pt:start_pt + config.window_size]
      start_pt += config.stride
      total_data.append(data)
    return np.array(total_data)

def sliding_window(arr):
    start_pt = 0
    total_data = []
    while(True):
      if len(arr) < (start_pt + config.window_size) : break
      data = arr[start_pt:start_pt + config.window_size]
      start_pt += config.stride
      total_data.append(data)
    return np.array(total_data)

def cutting_sliding_window(arr, window_size, stride) :
  count = math.floor((len(arr) - window_size) / stride); print("count", count)
  new_arr = arr[ : (count * stride + window_size)]
  return new_arr

def seed_everything(seed: int = 42):
  random.seed(seed)
  np.random.seed(seed)
  os.environ["PYTHONHASHSEED"] = str(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)  # type: ignore
  torch.backends.cudnn.deterministic = True  # type: ignore
  torch.backends.cudnn.benchmark = True  # type: ignore

def get_anomaly_time(original, prediction) : 
  temp = np.zeros(shape=(len(original),), dtype=np.float32)
  original = original.squeeze(axis = 1)

  for i in range(len(prediction)) :
    if prediction[i] == 0 :
      temp[i*config.stride : (i*config.stride + config.window_size)] = np.nan

    elif prediction[i] == 1 : # anomaly
      temp[i*config.stride : (i*config.stride + config.window_size)] = original[i*config.stride : (i*config.stride + config.window_size)]

  return temp

def drawing(pred, x) :
  #---# Drawing - 22 #---#
  plt.figure(figsize=(30,5))
  plt.plot(x, markersize=1)
  plt.plot(pred, marker='.', markersize=2, color='r', linestyle='None')

  #---# 실제 anomaly 값 구간 #---#
  a = np.linspace(62200, 65300)
  # plt.fill_between(a, 0, 2000, color='green', alpha=0.3)
  plt.fill_between(a, -1, 4, color='green', alpha=0.5)
  b = np.linspace(95600, 99300)
  # plt.fill_between(b, 0, 2000, color='green', alpha=0.5)
  plt.fill_between(b, -1, 4, color='green', alpha=0.5)
  c = np.linspace(148400, 152300)
  # plt.fill_between(c, 0, 2000, color='green', alpha=0.5)
  plt.fill_between(c, -1, 4, color='green', alpha=0.5)

  plt.show()
  plt.clf()

def calculate(true_list, pred_list) : 
  pred_list = pred_list.dropna()

  pred_anomaly_set = set(pred_list.index.tolist())
  pred_normal_set = set(range(len(true_list))) - pred_anomaly_set
  true_anomaly_set = set(np.where(np.array(true_list) != 0)[0].tolist())
  true_normal_set = set(np.where(np.array(true_list) == 0)[0].tolist())

  recall = len(pred_anomaly_set.intersection(true_anomaly_set)) / len(true_anomaly_set)
  accuracy = len(pred_anomaly_set.intersection(true_anomaly_set)) / len(pred_anomaly_set.union(true_anomaly_set)) # len(anomaly_set.union(true_set))
  # accuracy = (len(pred_anomaly_set.intersection(true_anomaly_set)) + len(pred_normal_set.intersection(true_normal_set))) / len(true_list) # (빨간 거 맞은거 + 파란거 맞은거) / 전체
  return recall, accuracy

In [56]:
import tensorflow as tf
from tensorflow.keras import Model ,models, layers, optimizers, utils



# reshape input into [samples, timesteps, features]
timesteps = 1000
n_features = 1
print("sequence shape", sequence.shape)
sequence = sequence.reshape((-1, timesteps, n_features))
print("shape", sequence.shape)
# sequence = sequence.reshape((n_in, 1000, 1))

# define model
model = models.Sequential()
model.add(layers.LSTM(100, activation='relu', input_shape=(timesteps, n_features)))
model.add(layers.RepeatVector(timesteps))
model.add(layers.LSTM(100, activation='relu', return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(n_features)))
model.compile(optimizer='adam', loss='mse')

model.summary()

# fit model
model.fit(sequence, sequence, epochs=300, verbose=0)

# predict
yhat = model.predict(sequence)
yhat

sequence shape (85, 1000)
shape (85, 1000, 1)
Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_50 (LSTM)              (None, 100)               40800     
                                                                 
 repeat_vector_19 (RepeatVec  (None, 1000, 100)        0         
 tor)                                                            
                                                                 
 lstm_51 (LSTM)              (None, 1000, 100)         80400     
                                                                 
 time_distributed_19 (TimeDi  (None, 1000, 1)          101       
 stributed)                                                      
                                                                 
Total params: 121,301
Trainable params: 121,301
Non-trainable params: 0
_________________________________________________________________


KeyboardInterrupt: ignored

In [ ]:
config = easydict.EasyDict({
    "num_epochs" : 2, #500
    "batch_size" : 16, #16 
    "mode" : 'train',
    # "mode" : "test",
    "lr" : 1e-3, 
    "wd" : None,
    "window_size" : 1000,
    "stride" : 250,
    "threshold" : 0.3, # 0.3이나 0.2로 하기
    "seed" : 1004,
    "device" : torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
})
seed_everything(config.seed)
tm = time.localtime(time.time())
string = time.strftime('%Y%m%d_%H%M%S', tm)

# wandb.init(project="Anomaly-Oil", entity="sohyun", name=string, magic=True)




# _, losses = predict(model, x_test_scaled)
# sns.distplot(losses, bins=50, kde=True)


# optimizer = torch.optim.Adam(params = model.parameters(), lr = config.lr) # lr = config.lr
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, threshold_mode='abs', min_lr=1e-8, verbose=True)

##########################
#---# true list 만들기 #---#
##########################
true_label = [0 for i in range(len(test_scale))] # test_scale
true_label[62200:65300] = [1 for i in range(62200,65300)]
true_label[95600:99300] = [1 for i in range(95600,99300)]
true_label[148400:152300] = [1 for i in range(148400,152300)]

true_label_sliding = sliding_window(np.array(true_label), config.window_size, config.stride)
true_label_sliding = np.expand_dims(np.array(true_label_sliding), 2) # dimension expansion

In [ ]:
ttrain.shape

AttributeError: ignored

In [ ]:
"""
Time Series error calculation functions.
"""

import math

import numpy as np
import pandas as pd
from pyts.metrics import dtw
from scipy import integrate


def regression_errors(y, y_hat, smoothing_window=0.01, smooth=True):
  """Compute an array of absolute errors comparing predictions and expected output.
  If smooth is True, apply EWMA to the resulting array of errors.
  Args:
    y (ndarray):
      Ground truth.
    y_hat (ndarray):
      Predicted values.
    smoothing_window (float):
      Optional. Size of the smoothing window, expressed as a proportion of the total
      length of y. If not given, 0.01 is used.
    smooth (bool):
      Optional. Indicates whether the returned errors should be smoothed with EWMA.
      If not given, `True` is used.
  Returns:
    ndarray:
      Array of errors.
  """
  errors = np.abs(y - y_hat)[:, 0]

  if not smooth:
    return errors

  smoothing_window = int(smoothing_window * len(y))

  return pd.Series(errors).ewm(span=smoothing_window).mean().values


def _point_wise_error(y, y_hat):
  """Compute point-wise error between predicted and expected values.
  The computed error is calculated as the difference between predicted
  and expected values with a rolling smoothing factor.
  Args:
    y (ndarray):
      Ground truth.
    y_hat (ndarray):
      Predicted values.
  Returns:
    ndarray:
      An array of smoothed point-wise error.
  """
  return abs(y - y_hat)


def _area_error(y, y_hat, score_window=10):
  """Compute area error between predicted and expected values.
  The computed error is calculated as the area difference between predicted
  and expected values with a smoothing factor.
  Args:
    y (ndarray):
      Ground truth.
    y_hat (ndarray):
      Predicted values.
    score_window (int):
      Optional. Size of the window over which the scores are calculated.
      If not given, 10 is used.
  Returns:
    ndarray:
      An array of area error.
  """
  smooth_y = pd.Series(y).rolling(
      score_window, center=True, min_periods=score_window // 2).apply(integrate.trapz)
  smooth_y_hat = pd.Series(y_hat).rolling(
      score_window, center=True, min_periods=score_window // 2).apply(integrate.trapz)

  errors = abs(smooth_y - smooth_y_hat)

  return errors


def _dtw_error(y, y_hat, score_window=10):
  """Compute dtw error between predicted and expected values.
  The computed error is calculated as the dynamic time warping distance
  between predicted and expected values with a smoothing factor.
  Args:
    y (ndarray):
      Ground truth.
    y_hat (ndarray):
      Predicted values.
    score_window (int):
      Optional. Size of the window over which the scores are calculated.
      If not given, 10 is used.
  Returns:
    ndarray:
      An array of dtw error.
  """
  length_dtw = (score_window // 2) * 2 + 1
  half_length_dtw = length_dtw // 2

  # add padding
  y_pad = np.pad(y, (half_length_dtw, half_length_dtw),
                  'constant', constant_values=(0, 0))
  y_hat_pad = np.pad(y_hat, (half_length_dtw, half_length_dtw),
                      'constant', constant_values=(0, 0))

  i = 0
  similarity_dtw = list()
  while i < len(y) - length_dtw:
    true_data = y_pad[i:i + length_dtw]
    true_data = true_data.flatten()

    pred_data = y_hat_pad[i:i + length_dtw]
    pred_data = pred_data.flatten()

    dist = dtw(true_data, pred_data)
    similarity_dtw.append(dist)
    i += 1

  errors = ([0] * half_length_dtw + similarity_dtw +
            [0] * (len(y) - len(similarity_dtw) - half_length_dtw))

  return errors


def reconstruction_errors(y, y_hat, step_size=1, score_window=10, smoothing_window=0.01,
                        smooth=True, rec_error_type='point'):
  """Compute an array of reconstruction errors.
  Compute the discrepancies between the expected and the
  predicted values according to the reconstruction error type.
  Args:
    y (ndarray):
      Ground truth.
    y_hat (ndarray):
      Predicted values. Each timestamp has multiple predictions.
    step_size (int):
      Optional. Indicating the number of steps between windows in the predicted values.
      If not given, 1 is used.
    score_window (int):
      Optional. Size of the window over which the scores are calculated.
      If not given, 10 is used.
    smoothing_window (float or int):
      Optional. Size of the smoothing window, when float it is expressed as a proportion
      of the total length of y. If not given, 0.01 is used.
    smooth (bool):
      Optional. Indicates whether the returned errors should be smoothed.
      If not given, `True` is used.
    rec_error_type (str):
      Optional. Reconstruction error types ``["point", "area", "dtw"]``.
      If not given, "point" is used.
  Returns:
    ndarray:
      Array of reconstruction errors.
  """
  if isinstance(smoothing_window, float):
    smoothing_window = min(math.trunc(len(y) * smoothing_window), 200)

  true = [item[0] for item in y.reshape((y.shape[0], -1))]
  for item in y[-1][1:]:
    true.extend(item)

  predictions = []
  predictions_vs = []

  pred_length = y_hat.shape[1]
  num_errors = y_hat.shape[1] + step_size * (y_hat.shape[0] - 1)

  for i in range(num_errors):
    intermediate = []
    for j in range(max(0, i - num_errors + pred_length), min(i + 1, pred_length)):
      intermediate.append(y_hat[i - j, j])
    if intermediate:
      predictions.append(np.median(np.asarray(intermediate)))

      predictions_vs.append([[
          np.min(np.asarray(intermediate)),
          np.percentile(np.asarray(intermediate), 25),
          np.percentile(np.asarray(intermediate), 50),
          np.percentile(np.asarray(intermediate), 75),
          np.max(np.asarray(intermediate))
      ]])

  true = np.asarray(true)
  predictions = np.asarray(predictions)
  predictions_vs = np.asarray(predictions_vs)

  # Compute reconstruction errors
  if rec_error_type.lower() == "point":
    errors = _point_wise_error(true, predictions)

  elif rec_error_type.lower() == "area":
    errors = _area_error(true, predictions, score_window)

  elif rec_error_type.lower() == "dtw":
    errors = _dtw_error(true, predictions, score_window)

  # Apply smoothing
  if smooth:
    errors = pd.Series(errors).rolling(
      smoothing_window, center=True, min_periods=smoothing_window // 2).mean().values

  return errors, predictions_vs

In [ ]:
"""
Time Series anomaly detection functions.
Some of the implementation is inspired by the paper https://arxiv.org/pdf/1802.04431.pdf
"""

import numpy as np
import pandas as pd
from scipy.optimize import fmin


def deltas(errors, epsilon, mean, std):
  """Compute mean and std deltas.
  delta_mean = mean(errors) - mean(all errors below epsilon)
  delta_std = std(errors) - std(all errors below epsilon)
  Args:
    errors (ndarray):
      Array of errors.
    epsilon (ndarray):
      Threshold value.
    mean (float):
      Mean of errors.
    std (float):
      Standard deviation of errors.
  Returns:
    float, float:
      * delta_mean.
      * delta_std.
  """
  below = errors[errors <= epsilon]
  if not len(below):
    return 0, 0

  return mean - below.mean(), std - below.std()


def count_above(errors, epsilon):
  """Count number of errors and continuous sequences above epsilon.
  Continuous sequences are counted by shifting and counting the number
  of positions where there was a change and the original value was true,
  which means that a sequence started at that position.
  Args:
    errors (ndarray):
      Array of errors.
    epsilon (ndarray):
      Threshold value.
  Returns:
    int, int:
      * Number of errors above epsilon.
      * Number of continuous sequences above epsilon.
  """
  above = errors > epsilon
  total_above = len(errors[above])

  above = pd.Series(above)
  shift = above.shift(1)
  change = above != shift

  total_consecutive = sum(above & change)

  return total_above, total_consecutive


def z_cost(z, errors, mean, std):
  """Compute how bad a z value is.
  The original formula is::
                (delta_mean/mean) + (delta_std/std)
      ------------------------------------------------------
      number of errors above + (number of sequences above)^2
  which computes the "goodness" of `z`, meaning that the higher the value
  the better the `z`.
  In this case, we return this value inverted (we make it negative), to convert
  it into a cost function, as later on we will use scipy.fmin to minimize it.
  Args:
    z (ndarray):
      Value for which a cost score is calculated.
    errors (ndarray):
      Array of errors.
    mean (float):
      Mean of errors.
    std (float):
      Standard deviation of errors.
  Returns:
    float:
      Cost of z.
  """
  epsilon = mean + z * std

  delta_mean, delta_std = deltas(errors, epsilon, mean, std)
  above, consecutive = count_above(errors, epsilon)

  numerator = -(delta_mean / mean + delta_std / std)
  denominator = above + consecutive ** 2

  if denominator == 0:
    return np.inf

  return numerator / denominator


def _find_threshold(errors, z_range):
  """Find the ideal threshold.
  The ideal threshold is the one that minimizes the z_cost function. Scipy.fmin is used
  to find the minimum, using the values from z_range as starting points.
  Args:
    errors (ndarray):
      Array of errors.
    z_range (list):
      List of two values denoting the range out of which the start points for the
      scipy.fmin function are chosen.
  Returns:
    float:
      Calculated threshold value.
  """
  mean = errors.mean()
  std = errors.std()

  min_z, max_z = z_range
  best_z = min_z
  best_cost = np.inf
  for z in range(min_z, max_z):
    best = fmin(z_cost, z, args=(errors, mean, std), full_output=True, disp=False)
    z, cost = best[0:2]
    if cost < best_cost:
      best_z = z[0]

  return mean + best_z * std


def _fixed_threshold(errors, k=4):
  """Calculate the threshold.
  The fixed threshold is defined as k standard deviations away from the mean.
  Args:
    errors (ndarray):
      Array of errors.
  Returns:
    float:
      Calculated threshold value.
  """
  mean = errors.mean()
  std = errors.std()

  return mean + k * std


def _find_sequences(errors, epsilon, anomaly_padding):
  """Find sequences of values that are above epsilon.
  This is done following this steps:
    * create a boolean mask that indicates which values are above epsilon.
    * mark certain range of errors around True values with a True as well.
    * shift this mask by one place, filing the empty gap with a False.
    * compare the shifted mask with the original one to see if there are changes.
    * Consider a sequence start any point which was true and has changed.
    * Consider a sequence end any point which was false and has changed.
  Args:
    errors (ndarray):
      Array of errors.
    epsilon (float):
      Threshold value. All errors above epsilon are considered an anomaly.
    anomaly_padding (int):
      Number of errors before and after a found anomaly that are added to the
      anomalous sequence.
  Returns:
    ndarray, float:
      * Array containing start, end of each found anomalous sequence.
      * Maximum error value that was not considered an anomaly.
  """
  above = pd.Series(errors > epsilon)
  index_above = np.argwhere(above.values)

  for idx in index_above.flatten():
    above[max(0, idx - anomaly_padding):min(idx + anomaly_padding + 1, len(above))] = True

  shift = above.shift(1).fillna(False)
  change = above != shift

  if above.all():
    max_below = 0
  else:
    max_below = max(errors[~above])

  index = above.index
  starts = index[above & change].tolist()
  ends = (index[~above & change] - 1).tolist()

  if len(ends) == len(starts) - 1:
    ends.append(len(above) - 1)

  return np.array([starts, ends]).T, max_below


def _get_max_errors(errors, sequences, max_below):
  """Get the maximum error for each anomalous sequence.
  Also add a row with the max error which was not considered anomalous.
  Table containing a ``max_error`` column with the maximum error of each
  sequence and the columns ``start`` and ``stop`` with the corresponding start and stop
  indexes, sorted descendingly by the maximum error.
  Args:
    errors (ndarray):
      Array of errors.
    sequences (ndarray):
      Array containing start, end of anomalous sequences
    max_below (float):
      Maximum error value that was not considered an anomaly.
  Returns:
    pandas.DataFrame:
      DataFrame object containing columns ``start``, ``stop`` and ``max_error``.
  """
  max_errors = [{
    'max_error': max_below,
    'start': -1,
    'stop': -1
  }]

  for sequence in sequences:
    start, stop = sequence
    sequence_errors = errors[start: stop + 1]
    max_errors.append({
        'start': start,
        'stop': stop,
        'max_error': max(sequence_errors)
    })

  max_errors = pd.DataFrame(max_errors).sort_values('max_error', ascending=False)
  return max_errors.reset_index(drop=True)


def _prune_anomalies(max_errors, min_percent):
  """Prune anomalies to mitigate false positives.
  This is done by following these steps:
    * Shift the errors 1 negative step to compare each value with the next one.
    * Drop the last row, which we do not want to compare.
    * Calculate the percentage increase for each row.
    * Find rows which are below ``min_percent``.
    * Find the index of the latest of such rows.
    * Get the values of all the sequences above that index.
  Args:
    max_errors (pandas.DataFrame):
      DataFrame object containing columns ``start``, ``stop`` and ``max_error``.
    min_percent (float):
      Percentage of separation the anomalies need to meet between themselves and the
      highest non-anomalous error in the window sequence.
  Returns:
    ndarray:
      Array containing start, end, max_error of the pruned anomalies.
  """
  next_error = max_errors['max_error'].shift(-1).iloc[:-1]
  max_error = max_errors['max_error'].iloc[:-1]

  increase = (max_error - next_error) / max_error
  too_small = increase < min_percent

  if too_small.all():
    last_index = -1
  else:
    last_index = max_error[~too_small].index[-1]

  return max_errors[['start', 'stop', 'max_error']].iloc[0: last_index + 1].values


def _compute_scores(pruned_anomalies, errors, threshold, window_start):
  """Compute the score of the anomalies.
  Calculate the score of the anomalies proportional to the maximum error in the sequence
  and add window_start timestamp to make the index absolute.
  Args:
    pruned_anomalies (ndarray):
      Array of anomalies containing the start, end and max_error for all anomalies in
        the window.
      errors (ndarray):
        Array of errors.
      threshold (float):
        Threshold value.
      window_start (int):
        Index of the first error value in the window.
  Returns:
    list:
      List of anomalies containing start-index, end-index, score for each anomaly.
  """
  anomalies = list()
  denominator = errors.mean() + errors.std()

  for row in pruned_anomalies:
    max_error = row[2]
    score = (max_error - threshold) / denominator
    anomalies.append([row[0] + window_start, row[1] + window_start, score])

  return anomalies


def _merge_sequences(sequences):
  """Merge consecutive and overlapping sequences.
  We iterate over a list of start, end, score triples and merge together
  overlapping or consecutive sequences.
  The score of a merged sequence is the average of the single scores,
  weighted by the length of the corresponding sequences.
  Args:
    sequences (list):
      List of anomalies, containing start-index, end-index, score for each anomaly.
  Returns:
    ndarray:
      Array containing start-index, end-index, score for each anomaly after merging.
  """
  if len(sequences) == 0:
    return np.array([])

  sorted_sequences = sorted(sequences, key=lambda entry: entry[0])
  new_sequences = [sorted_sequences[0]]
  score = [sorted_sequences[0][2]]
  weights = [sorted_sequences[0][1] - sorted_sequences[0][0]]

  for sequence in sorted_sequences[1:]:
    prev_sequence = new_sequences[-1]

    if sequence[0] <= prev_sequence[1] + 1:
      score.append(sequence[2])
      weights.append(sequence[1] - sequence[0])
      weighted_average = np.average(score, weights=weights)
      new_sequences[-1] = (prev_sequence[0], max(prev_sequence[1], sequence[1]),
                            weighted_average)
    else:
      score = [sequence[2]]
      weights = [sequence[1] - sequence[0]]
      new_sequences.append(sequence)

  return np.array(new_sequences)


def _find_window_sequences(window, z_range, anomaly_padding, min_percent, window_start,
                           fixed_threshold):
  """Find sequences of values that are anomalous.
  We first find the threshold for the window, then find all sequences above that threshold.
  After that, we get the max errors of the sequences and prune the anomalies. Lastly, the
  score of the anomalies is computed.
  Args:
    window (ndarray):
      Array of errors in the window that is analyzed.
    z_range (list):
      List of two values denoting the range out of which the start points for the
      dynamic find_threshold function are chosen.
    anomaly_padding (int):
      Number of errors before and after a found anomaly that are added to the anomalous
      sequence.
    min_percent (float):
      Percentage of separation the anomalies need to meet between themselves and the
      highest non-anomalous error in the window sequence.
    window_start (int):
      Index of the first error value in the window.
    fixed_threshold (bool):
      Indicates whether to use fixed threshold or dynamic threshold.
  Returns:
    ndarray:
      Array containing the start-index, end-index, score for each anomalous sequence
      that was found in the window.
  """
  if fixed_threshold:
    threshold = _fixed_threshold(window)

  else:
    threshold = _find_threshold(window, z_range)

  window_sequences, max_below = _find_sequences(window, threshold, anomaly_padding)
  max_errors = _get_max_errors(window, window_sequences, max_below)
  pruned_anomalies = _prune_anomalies(max_errors, min_percent)
  window_sequences = _compute_scores(pruned_anomalies, window, threshold, window_start)

  return window_sequences


def find_anomalies(errors, index, z_range=(0, 10), window_size=None, window_size_portion=None,
                   window_step_size=None, window_step_size_portion=None, min_percent=0.1,
                   anomaly_padding=50, lower_threshold=False, fixed_threshold=None):
  """Find sequences of error values that are anomalous.
  We first define the window of errors, that we want to analyze. We then find the anomalous
  sequences in that window and store the start/stop index pairs that correspond to each
  sequence, along with its score. Optionally, we can flip the error sequence around the mean
  and apply the same procedure, allowing us to find unusually low error sequences.
  We then move the window and repeat the procedure.
  Lastly, we combine overlapping or consecutive sequences.
  Args:
    errors (ndarray):
      Array of errors.
    index (ndarray):
      Array of indices of the errors.
    z_range (list):
      Optional. List of two values denoting the range out of which the start points for
      the scipy.fmin function are chosen. If not given, (0, 10) is used.
    window_size (int):
      Optional. Size of the window for which a threshold is calculated. If not given,
      `None` is used, which finds one threshold for the entire sequence of errors.
    window_size_portion (float):
      Optional. Specify the size of the window to be a portion of the sequence of errors.
      If not given, `None` is used, and window size is used as is.
    window_step_size (int):
      Optional. Number of steps the window is moved before another threshold is
      calculated for the new window.
    window_step_size_portion (float):
      Optional. Specify the number of steps to be a portion of the window size. If not given,
      `None` is used, and window step size is used as is.
    min_percent (float):
      Optional. Percentage of separation the anomalies need to meet between themselves and
      the highest non-anomalous error in the window sequence. It nof given, 0.1 is used.
    anomaly_padding (int):
      Optional. Number of errors before and after a found anomaly that are added to the
      anomalous sequence. If not given, 50 is used.
    lower_threshold (bool):
      Optional. Indicates whether to apply a lower threshold to find unusually low errors.
      If not given, `False` is used.
    fixed_threshold (bool):
      Optional. Indicates whether to use fixed threshold or dynamic threshold. If not
      given, `False` is used.
  Returns:
    ndarray:
      Array containing start-index, end-index, score for each anomalous sequence that
      was found.
  """
  window_size = window_size or len(errors)
  if window_size_portion:
    window_size = np.ceil(len(errors) * window_size_portion).astype('int')

  window_step_size = window_step_size or window_size
  if window_step_size_portion:
    window_step_size = np.ceil(window_size * window_step_size_portion).astype('int')

  window_start = 0
  window_end = 0
  sequences = list()

  while window_end < len(errors):
    window_end = window_start + window_size
    window = errors[window_start:window_end]
    window_sequences = _find_window_sequences(window, z_range, anomaly_padding, min_percent,
                                              window_start, fixed_threshold)
    sequences.extend(window_sequences)

    if lower_threshold:
      # Flip errors sequence around mean
      mean = window.mean()
      inverted_window = mean - (window - mean)
      inverted_window_sequences = _find_window_sequences(inverted_window, z_range,
                                                          anomaly_padding, min_percent,
                                                          window_start, fixed_threshold)
      sequences.extend(inverted_window_sequences)

    window_start = window_start + window_step_size

  sequences = _merge_sequences(sequences)

  anomalies = list()

  for start, stop, score in sequences:
    anomalies.append([index[int(start)], index[int(stop)], score])

  return np.asarray(anomalies)